In [1]:
import datetime
import requests
import pandas as pd
import time
import glob

In [2]:
# format date

date = datetime.date(2019, 3, 28)
end_date = datetime.date(2019, 11, 1)
site = 20 # draftkings
sport_ids = {
    'mlb': 2,
    'nhl': 4,
    'nba': 3
}

url = 'https://resultsdb-api.rotogrinders.com/api/slates?start={}&site={}'.format(date.strftime("%m/%d/%Y"), site)

headers = {
    # ':authority': 'resultsdb-api.rotogrinders.com',
    # ':method': 'GET',
    # ':path': '/api/slates?start={}&site={}'.format(date.strftime("%m/%d/%Y"), site),
    # ':scheme': 'https',
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9',
    'if-none-match': 'W/"c3bac-w6jnMRcyrrcG6rXwd9uXH5jC57Y"',
    'origin': 'https://rotogrinders.com',
    'referer': 'https://rotogrinders.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
    'sec-ch-ua-mobile': '?0',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
}

In [3]:
summaries = []

while date < end_date:
  print(date.strftime("%m/%d/%Y"))
  url = 'https://resultsdb-api.rotogrinders.com/api/slates?start={}&site={}'.format(date.strftime("%m/%d/%Y"), site)
  response = requests.get(url = url, headers = headers)

  if response.status_code != 200:
    print(response.status_code, response.reason)

  mlb_games = [x for x in response.json() if x['sport'] == sport_ids['mlb'] and x['slateTypeName'] == 'Classic']

  if len(mlb_games) == 0:
    date = date + datetime.timedelta(days=1)
    continue

  main_slate = max(mlb_games, key=lambda x:len(x['slatePlayers']))
  players_df = pd.DataFrame.from_records(main_slate['slatePlayers'])
  games_df = pd.json_normalize(main_slate['slateGames'])
  # display(players_df['teamId'].unique())
  # display(games_df)
  full_df = pd.concat([
                     pd.merge(players_df, games_df, left_on='teamId', right_on='teamAway.id'),
                     pd.merge(players_df, games_df, left_on='teamId', right_on='teamHome.id')
  ], ignore_index=True)
  full_df.to_csv(date.strftime("%m-%d-%Y.csv"))

  summary_url = 'https://resultsdb-api.rotogrinders.com/api/slates/{}/summary'.format(main_slate['_id'])
  summary_response = requests.get(url = summary_url, headers = headers)
  if summary_response.status_code != 200:
    print(summary_url, summary_response.status_code, summary_response.reason)
    date = date + datetime.timedelta(days=1)
    continue

  summary = summary_response.json()
  summary.pop('topEntries', None)
  summary.pop('winnerMap', None)
  summary.pop('topScoringEntry', None)
  summary['date'] = date.strftime("%m/%d/%Y")

  summaries.append(summary)

  date = date + datetime.timedelta(days=1)

03/28/2019
03/29/2019
03/30/2019
03/31/2019
04/01/2019
04/02/2019
04/03/2019
04/04/2019
04/05/2019
04/06/2019
04/07/2019
04/08/2019
04/09/2019
04/10/2019
04/11/2019
04/12/2019
04/13/2019
04/14/2019
04/15/2019
04/16/2019
04/17/2019
https://resultsdb-api.rotogrinders.com/api/slates/5bf6ff9ef00b7d0c057c8449/summary 500 Internal Server Error
04/18/2019
04/19/2019
04/20/2019
04/21/2019
04/22/2019
04/23/2019
04/24/2019
04/25/2019
04/26/2019
04/27/2019
04/28/2019
04/29/2019
04/30/2019
05/01/2019
05/02/2019
05/03/2019
05/04/2019
05/05/2019
05/06/2019
05/07/2019
05/08/2019
https://resultsdb-api.rotogrinders.com/api/slates/5bf6f4229e72870c0672d67f/summary 500 Internal Server Error
05/09/2019
05/10/2019
05/11/2019
05/12/2019
05/13/2019
05/14/2019
05/15/2019
05/16/2019
05/17/2019
05/18/2019
05/19/2019
05/20/2019
05/21/2019
05/22/2019
05/23/2019
05/24/2019
05/25/2019
05/26/2019
05/27/2019
05/28/2019
05/29/2019
05/30/2019
05/31/2019
06/01/2019
06/02/2019
06/03/2019
06/04/2019
06/05/2019
06/06/2019
0

In [4]:
all_files = glob.glob("*.csv")
season_df = pd.concat((pd.read_csv(f) for f in all_files))

In [5]:
season_df

,Unnamed: 0,_id,teamId,opponent,scheduleId,rgPlayerId,team,salary,slatePosition,siteSlatePlayerId,dkName,name,statId,projectedFpts,projectedOwnership,weather,date,sport_id,id,vegas.props.strikeouts,vegas.movement,vegas.line,vegas.total,vegas.opp_total,vegas.o/u,teamAway.hashtag,teamAway.name,teamAway.id,teamHome.hashtag,teamHome.name,teamHome.id,weather.color,weather.description,ecr,vegas.props.home runs,vegas.props.hits,vegas.props.rbis,vegas
0,0,5d389683e2699f722dbcf6e9,98.0,TOR,57391,34646.0,CLE,12100,P,13010065,Mike Clevinger,Mike Clevinger,NaN,NaN,NaN,NaN,2019-07-23T23:07:00.000Z,2,57391,3.5 (-177),-0.27,160.0,3.81,5.19,9.0,CLE,98,98,TOR,108,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5d389683e2699f722dbcf6e7,98.0,TOR,57391,13394.0,CLE,11400,P,13008880,Trevor Bauer,Trevor Bauer,734454.0,22.72,0.1,NaN,2019-07-23T23:07:00.000Z,2,57391,3.5 (-177),-0.27,160.0,3.81,5.19,9.0,CLE,98,98,TOR,108,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5d389683e2699f722dbcf6d7,98.0,TOR,57391,351838.0,CLE,10400,P,13010080,Shane Bieber,Shane Bieber,NaN,NaN,NaN,NaN,2019-07-23T23:07:00.000Z,2,57391,3.5 (-177),-0.27,160.0,3.81,5.19,9.0,CLE,98,98,TOR,108,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5d389682e2699f722dbcf6bc,98.0,TOR,57391,NaN,CLE,9400,P,13010102,Corey Kluber,Corey Kluber,NaN,NaN,NaN,NaN,2019-07-23T23:07:00.000Z,2,57391,3.5 (-177),-0.27,160.0,3.81,5.19,9.0,CLE,98,98,TOR,108,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5d389682e2699f722dbcf6ad,98.0,TOR,57391,10678.0,CLE,9000,P,13010116,Carlos Carrasco,Carlos Carrasco,NaN,NaN,NaN,NaN,2019-07-23T23:07:00.000Z,2,57391,3.5 (-177),-0.27,160.0,3.81,5.19,9.0,CLE,98,98,TOR,108,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,1092,5d1b69ef6c89381921260252,96.0,CLE,57122,18804.0,BAL,2300,1B,12894022,Chris Davis,Chris Davis,727437.0,6.63,0.1,NaN,2019-06-30T17:05:00.000Z,2,57122,6.5 (+110),0.32,-270.0,6.68,3.82,10.5,CLE,98,98,BAL,96,96,green,"Temp around 90, winds blowing across the ballp...",NaN,NaN,NaN,NaN,NaN
1093,1093,5d1b69ef6c89381921260250,96.0,CLE,57122,NaN,BAL,2300,3B,12894048,Jomar Reyes,Jomar Reyes,NaN,NaN,NaN,NaN,2019-06-30T17:05:00.000Z,2,57122,6.5 (+110),0.32,-270.0,6.68,3.82,10.5,CLE,98,98,BAL,96,96,green,"Temp around 90, winds blowing across the ballp...",NaN,NaN,NaN,NaN,NaN
1094,1094,5d1b69ef6c8938192126023e,96.0,CLE,57122,NaN,BAL,2200,2B,12894009,Christopher Bostick,Christopher Bostick,NaN,NaN,NaN,NaN,2019-06-30T17:05:00.000Z,2,57122,6.5 (+110),0.32,-270.0,6.68,3.82,10.5,CLE,98,98,BAL,96,96,green,"Temp around 90, winds blowing across the ballp...",NaN,NaN,NaN,NaN,NaN
1095,1095,5d1b69ef6c8938192126023d,96.0,CLE,57122,813824.0,BAL,2200,SS,12894041,Richie Martin,Richie Martin,NaN,NaN,NaN,NaN,2019-06-30T17:05:00.000Z,2,57122,6.5 (+110),0.32,-270.0,6.68,3.82,10.5,CLE,98,98,BAL,96,96,green,"Temp around 90, winds blowing across the ballp...",NaN,NaN,NaN,NaN,NaN


In [6]:
season_df.to_csv(date.strftime("resultsdb_draftkings_mlb_%Y.csv"))

In [7]:
# summaries = []

# while date < end_date:
#   print(date.strftime("%m/%d/%Y"))
#   response = requests.get(url = url, headers = headers)

#   if response.status_code != 200:
#     print(response.status_code, response.reason)

#   mlb_games = [x for x in response.json() if x['sport'] == sport_ids['mlb'] and x['slateTypeName'] == 'Classic']
#   main_slate = max(mlb_games, key=lambda x:len(x['slatePlayers']))

#   summary_url = 'https://resultsdb-api.rotogrinders.com/api/slates/{}/summary'.format(main_slate['_id'])
#   summary_response = requests.get(url = summary_url, headers = headers)
#   if summary_response.status_code != 200:
#     print(summary_response.status_code, summary_response.reason)

#   summary = summary_response.json()
#   summary.pop('topEntries', None)
#   summary.pop('winnerMap', None)
#   summary.pop('topScoringEntry', None)
#   summary['date'] = date.strftime("%m/%d/%Y")

#   summaries.append(summary)

#   date = date + datetime.timedelta(days=1)
#   time.sleep(0.1)

# summaries_df = pd.concat((pd.json_normalize(s) for s in summaries), ignore_index=True)

In [9]:
summaries_df = pd.concat((pd.json_normalize(s) for s in summaries), ignore_index=True)
summaries_df

,cashAverage,gppAverage,optimalLineup,topEntry,date,bestHitter.title,bestHitter.value,bestPitcher.title,bestPitcher.value
0,131.312500,139.542308,"[{'ownership': '41.49%', 'position': 'P', 'per...","[{'fpts': 39.45, 'name': 'Jose Berrios', 'opp'...",03/28/2019,Best Hitter,Joc Pederson (39),Best Pitcher,Jose Berrios (39.45)
1,117.900000,128.296154,"[{'ownership': '64.29%', 'position': 'P', 'per...","[{'fpts': 25.9, 'name': 'German Marquez', 'opp...",03/29/2019,Best Hitter,Paul Goldschmidt (51),Best Pitcher,Matt Shoemaker (31.35)
2,83.607143,111.140000,"[{'ownership': '10.00%', 'position': 'P', 'per...","[{'fpts': 20.8, 'name': 'Kenta Maeda', 'opp': ...",03/30/2019,Best Hitter,Cody Bellinger (42),Best Pitcher,Brett Anderson (22.5)
3,78.500000,102.094118,"[{'ownership': '10.00%', 'position': 'P', 'per...","[{'fpts': 28.6, 'name': 'Lucas Giolito', 'opp'...",03/31/2019,Best Hitter,Trea Turner (39),Best Pitcher,Sandy Alcantara (31.6)
4,78.610000,94.065000,"[{'ownership': '10.00%', 'position': 'P', 'per...","[{'fpts': 22.85, 'name': 'Julio Urias', 'opp':...",04/01/2019,Best Hitter,Kevin Kiermaier (25),Best Pitcher,David Hess (33.65)
...,...,...,...,...,...,...,...,...,...
182,0.000000,108.200000,"[{'ownership': '65.36%', 'position': 'P', 'per...","[{'fpts': 26.55, 'name': 'Mike Soroka', 'opp':...",10/06/2019,Best Hitter,Russell Martin (29),Best Pitcher,Adam Wainwright (29.65)
183,80.266667,126.410000,"[{'ownership': '44.20%', 'position': 'P', 'per...","[{'fpts': 27.55, 'name': 'Max Scherzer', 'opp'...",10/07/2019,Best Hitter,Gleyber Torres (33),Best Pitcher,Charlie Morton (28.25)
184,84.300000,124.133333,"[{'ownership': '56.63%', 'position': 'P', 'per...","[{'fpts': 17.3, 'name': 'Stephen Strasburg', '...",10/09/2019,Best Hitter,Anthony Rendon (28),Best Pitcher,Jack Flaherty (27.9)
185,67.933333,113.883333,"[{'ownership': '74.63%', 'position': 'P', 'per...","[{'fpts': 39.95, 'name': 'Max Scherzer', 'opp'...",10/12/2019,Best Hitter,Gleyber Torres (30),Best Pitcher,Max Scherzer (39.95)


In [10]:
summaries_df.to_csv(date.strftime("resultsdb_draftkings_mlb_%Y_summaries.csv"))